In [1]:
!git clone https://github.com/samoletpanfilov/reinforcement_task.git
!git clone https://github.com/yursur/Samolet_hack.git

Cloning into 'reinforcement_task'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 18 (delta 3), reused 18 (delta 3), pack-reused 0
Unpacking objects: 100% (18/18), 278.35 KiB | 9.94 MiB/s, done.
Cloning into 'Samolet_hack'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 30 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (30/30), 4.52 MiB | 6.27 MiB/s, done.


In [2]:
import pandas as pd
import numpy as np

In [3]:
test = pd.read_excel('/content/reinforcement_task/data/test.xlsx')
test.head()

,dt,Цена на арматуру
0,2022-09-05,46000
1,2022-09-12,46000
2,2022-09-19,46000
3,2022-09-26,45500
4,2022-10-03,45500


In [4]:
!pip install darts
from darts import TimeSeries

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.9/693.9 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 18.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.6/572.6 kB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.2/721.2 kB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.

In [5]:
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel
from darts.metrics import mape
from darts.datasets import TimeSeries
from darts import concatenate
from sklearn.preprocessing import MaxAbsScaler
from darts.models import TCNModel, BlockRNNModel
from darts.utils.likelihood_models import GaussianLikelihood

best = BlockRNNModel.load('/content/Samolet_hack/best')
transformer = Scaler(MaxAbsScaler())

In [6]:
preds = best.predict(n=len(test.dt))
preds = preds.all_values()
preds = [elem for sublist in preds for subsublist in sublist for elem in subsublist]
test['pred'] = preds
test.head()

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: 0it [00:00, ?it/s]

,dt,Цена на арматуру,pred
0,2022-09-05,46000,0.527027
1,2022-09-12,46000,0.526480
2,2022-09-19,46000,0.524597
3,2022-09-26,45500,0.526792
4,2022-10-03,45500,0.526680


In [7]:
def decision_prices(test):

    test = test.set_index('dt')
    tender_price = test['Цена на арматуру']
    decision = test['Объем']
    start_date = test.index.min()
    end_date = test.index.max()

    _results = []
    _active_weeks = 0
    for report_date in pd.date_range(start_date, end_date, freq='W-MON'):
        if _active_weeks == 0:  # Пришла пора нового тендера
            _fixed_price = tender_price.loc[report_date]
            _active_weeks = int(decision.loc[report_date])
        _results.append(_fixed_price)
        _active_weeks += -1
    cost = sum(_results)
    return cost # Возвращаем затраты на периоде

In [8]:
def volume(df, name):
    price = df[name].values
    out = [0]*len(df)

    i = 0
    while i < len(df):
        i += 1
        count = 0
        for j in range(i+1, min(i+11, len(df))):
            count += 1
            if price[i] >= price[j]:
                out[i] = count
                i += count - 1
                count = 0
                break
        else:
            out[i] = min(10, len(df) - i)
            i += 9
            count = 0
    out[0] = 1
    df['Объем'] = out
    return df

In [9]:
decision_prices(volume(test, 'Цена на арматуру'))

1192500

In [10]:
decision_prices(volume(test, 'pred'))

1219000

In [11]:
test

,dt,Цена на арматуру,pred,Объем
0,2022-09-05,46000,0.527027,1
1,2022-09-12,46000,0.526480,1
2,2022-09-19,46000,0.524597,10
3,2022-09-26,45500,0.526792,0
4,2022-10-03,45500,0.526680,0
5,2022-10-10,45000,0.529520,0
6,2022-10-17,44000,0.530883,0
7,2022-10-24,43000,0.532356,0
8,2022-10-31,42000,0.532926,0
9,2022-11-07,41500,0.533161,0


In [12]:
test.drop('pred', axis=1).to_excel('submission.xlsx', index=False)